# Sinopia testing of `bibframe2marc.xsl`


Sets-up notebook environment and initializes three RDF namespaces and prints the current date.

In [1]:
import datetime
import io
import rdflib
import requests
import pymarc
import sys
from lxml import etree
print(datetime.datetime.utcnow())

BF = rdflib.Namespace("http://id.loc.gov/ontologies/bibframe/")
BFLC = rdflib.Namespace("http://id.loc.gov/ontologies/bflc/")
SINOPIA = rdflib.Namespace("http://sinopia.io/vocabulary/")



2019-12-16 16:45:22.908791


Function binds three of the specific RDF namespaces used by Sinopia.

In [2]:
def init_ns(graph):
    graph.namespace_manager.bind("bf", BF)
    graph.namespace_manager.bind("bflc", BFLC)
    graph.namespace_manager.bind("sinopia", SINOPIA)

In [3]:

# Function takes a list Sinopia URIs and returns a RDF Graph
def triples2record(uris):
    record_graph = rdflib.Graph()
    init_ns(record_graph)
    for uri in uris:
        get_request = requests.get(uri)
        raw_utf8 = get_request.text.encode(get_request.encoding).decode()
        record_graph.parse(data=raw_utf8, format='turtle')
    return record_graph

# Function takes a graph, serializes to XML, and the returns the transformed MARC XML
def graph2marcXML(graph):
    rdf_xml = etree.XML(graph.serialize(format='pretty-xml'))
    try:
        return bf2marc_transform(rdf_xml)
    except:
        print(f"Error transforming graph to {sys.exc_info()[0]}")
    
# Function takes a list of Sinopia URLs, creates graphs for each, serializes to XML before, combining
# back into a single XML document for transformation 
def graph2RDFXML(uris):
    for uri in uris:
        graph = triples2record(uri)
        rdf_xml = etree.XML(graph.serialize(format='pretty-xml'))
        yield rdf_xml

# Function takes a MARC XML document and returns the MARC 21 equalivant
def marcXMLto21(marcXML):
    reader = io.StringIO(etree.tostring(marcXML).decode())
    marc_records = pymarc.parse_xml_to_array(reader)
    # Should only be 1 record in the list
    return marc_records[0]
    
# Function takes a Sinopia URI that has a relative URI, replaces relative with absolute, and returns 
# new graph
def update_abs_url(uri):
    get_request = requests.get(uri)
    raw_utf8 = get_request.text.encode(get_request.encoding).decode()
    raw_utf8 = raw_utf8.replace('<>', f"<{uri}>")
    graph = rdflib.Graph()
    init_ns(graph)
    graph.parse(data=raw_utf8, format='turtle')
    return graph

Opens the XSLT as a file and creates an [lxml XSLT](https://lxml.de/xpathxslt.html) object.

In [4]:
with open("../src/xslt/bibframe2marc.xsl", "rb") as fo:
    xml = etree.XML(fo.read())
    bf2marc_transform = etree.XSLT(xml)

# Examples -- Stanford University
We are using a number of Stanford's BIBFRAME Instances and Works in Sinopia that use both nested and unnested resource templates. 

For some of the comparisions, we are using older resources that were created prior to the Sinopia PR#[1868](https://github.com/LD4P/sinopia_editor/pull/1868) that use relative urls (i.e. `<>`) instead of the resource's absolute URI. The `update_abs_url` function does a quick string replacement and returns a graph.

In [25]:
stanford_nested = update_abs_url('https://trellis.stage.sinopia.io/repository/stanford/d2d48e03-c87f-430d-8708-e8d3d8489105')
print(stanford_nested.serialize(format='turtle').decode())

@prefix acl: <http://www.w3.org/ns/auth/acl#> .
@prefix as: <https://www.w3.org/ns/activitystreams#> .
@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix memento: <http://mementoweb.org/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://trellis.stage.sinopia.io/reposi

In [47]:
stanford_nested_marc_xml = graph2marcXML(stanford_nested)
print(stanford_nested_marc_xml)
stanford_nested_marc21 = marcXMLto21(stanford_nested_marc_xml)
print(stanford_nested_marc21)

<?xml version="1.0"?>
<marc:record xmlns:marc="http://www.loc.gov/MARC21/slim" xmlns:fn="http://www.w3.org/2005/xpath-function">
  <marc:leader>     nam a22     uu 4500</marc:leader>
  <marc:controlfield tag="001">idp140491424692152</marc:controlfield>
  <marc:controlfield tag="003">DLC</marc:controlfield>
  <marc:controlfield tag="008">700101|||||||||xx            ||| |     |</marc:controlfield>
  <marc:datafield tag="020" ind1=" " ind2=" ">
    <marc:subfield code="a">9786077153511</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="040" ind1=" " ind2=" "/>
  <marc:datafield tag="245" xml:lang="es" ind1="1" ind2="0">
    <marc:subfield code="a">Semblanzas de la agricultura de las Américas</marc:subfield>
    <marc:subfield code="c">coordinación editorial, Víctor M. Villalobos Arámbula, Franklin Marín Vargas, Federico Sancho Guevara, J. Cruz García Albarado</marc:subfield>
    <marc:subfield code="c"/>
  </marc:datafield>
  <marc:datafield tag="246" xml:lang="en" ind1="1" ind2=

For the unnested testing, we use the same BF Instance (https://trellis.stage.sinopia.io/repository/stanford/e91d50ce-b863-457a-824e-7bd0f9c8a9e7) and tries first BF Work (https://trellis.stage.sinopia.io/repository/stanford/fbe7b462-2894-44c4-bd56-c8dde8c568a3) and displays the MARC 21 result for comparision. 

The BF Work (https://trellis.stage.sinopia.io/repository/stanford/fbe7b462-2894-44c4-bd56-c8dde8c568a3) has an incorrect predicate URL for of `http://bibframe.org/ontologies/Contribution` for the contribution class that results in the contribution fields to not transform correctly to the equalivant MARC fields (100, etc.) in the transformation.

In [8]:
stanford_instance = update_abs_url('https://trellis.stage.sinopia.io/repository/stanford/e91d50ce-b863-457a-824e-7bd0f9c8a9e7')
stanford_work1 =  update_abs_url('https://trellis.stage.sinopia.io/repository/stanford/fbe7b462-2894-44c4-bd56-c8dde8c568a3')
instance_xml = etree.XML(stanford_instance.serialize(format='pretty-xml'))
work1_xml = etree.XML(stanford_work1.serialize(format='pretty-xml'))
print(etree.tostring(work1_xml).decode())

<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:ns1="http://bibframe.org/ontologies/" xmlns:sinopia="http://sinopia.io/vocabulary/">
  <bf:Work rdf:about="https://trellis.stage.sinopia.io/repository/stanford/fbe7b462-2894-44c4-bd56-c8dde8c568a3">
    <bf:originDate xml:lang="en">2017</bf:originDate>
    <bf:contribution>
      <ns1:Contribution rdf:nodeID="f4e098454e8354072979bcc9aeb287943b5">
        <ns1:Contribution rdf:resource="http://id.loc.gov/vocabulary/relators/edt"/>
        <ns1:agent>Mar&#237;n Vargas, Franklin </ns1:agent>
      </ns1:Contribution>
    </bf:contribution>
    <bf:subject rdf:resource="http://id.loc.gov/authorities/subjects/sh85002415"/>
    <bf:geographicCoverage>America </bf:geographicCoverage>
    <bf:language rdf:resource="http://id.loc.gov/vocabulary/languages/spa"/>
    <bf:title>
      <bf:Title rdf:nodeID="f4e098454e8354072979bcc9aeb287943b4">
        <bf:mainTitle xml:lang="es"

In [39]:
stanford_unnested = update_abs_url('https://trellis.stage.sinopia.io/repository/stanford/e91d50ce-b863-457a-824e-7bd0f9c8a9e7')
# With first Work resource
stanford_unnested_wrk1 = stanford_unnested + update_abs_url('https://trellis.stage.sinopia.io/repository/stanford/fbe7b462-2894-44c4-bd56-c8dde8c568a3')
stanford_unnested_wrk1_marc_xml = graph2marcXML(stanford_unnested)
stanford_unnested_wrk1_marc_21 = marcXMLto21(stanford_unnested_wrk1_marc_xml)
print(stanford_unnested_wrk1_marc_21)

=LDR       nam a22     uu 4500
=001  idp140491395026712
=003  DLC
=008  19121310|||||||||xx\\\\\\\\\\\\|||\|\\\\\|
=020  \\$a9787502744878
=245  10$a中國古動物$c關键
=246  11$a中国古動物
=246  11$aThe fossil animals of China
=250  \\$a第1版
=264  \1$c1998.
=300  \\$a197 pages$c29 cm
=500  \\$aText in Chinese, English, and Japanese ; nomenclature also in Latin
=500  \\$aIncludes index in Latin
=884  \\$aDLC bibframe2marc-xsl v0.1.0-SNAPSHOT$g20191213104714.0$qDLC$uhttps://github.com/lcnetdev/bibframe2marc-xsl



Using the second Work the combined graph serializes correctly but is causing a XSLT error that the top-level BF Instance is missing even though the XML shows a top-level BF Instance.

In [45]:
stanford_unnested_wrk2 = stanford_unnested + update_abs_url('https://trellis.stage.sinopia.io/repository/stanford/c59fbd1c-e13c-47ec-8e90-5740473edd91')
print(stanford_unnested_wrk2.serialize(format='turtle').decode())
wrk2_rdf_xml = etree.XML(stanford_unnested_wrk2.serialize(format='pretty-xml'))
print(etree.tostring(wrk2_rdf_xml).decode())
stanford_unnested_wrk2_xml = bf2marc_transform(wrk2_rdf_xml)

@prefix acl: <http://www.w3.org/ns/auth/acl#> .
@prefix as: <https://www.w3.org/ns/activitystreams#> .
@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix memento: <http://mementoweb.org/ns#> .
@prefix ns1: <http://bibframe.org/ontologies/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSch

## Un-nested Work and Instance Examples
Sinopia work URL of 'https://trellis.stage.sinopia.io/repository/stanford/2344e198-4306-45eb-ac55-e8a01836820b' with a corresponding 
instance URL  = 'https://trellis.stage.sinopia.io/repository/stanford/63f532de-65d3-4da4-bae3-9e981bd12eaf'


In [9]:
stanford_graph = triples2record(['https://trellis.stage.sinopia.io/repository/stanford/2344e198-4306-45eb-ac55-e8a01836820b',
                                'https://trellis.stage.sinopia.io/repository/stanford/63f532de-65d3-4da4-bae3-9e981bd12eaf'])

In [10]:
marc_XML = graph2marcXML(stanford_graph)

In [11]:
print(marc_XML)

<?xml version="1.0"?>
<marc:record xmlns:marc="http://www.loc.gov/MARC21/slim" xmlns:fn="http://www.w3.org/2005/xpath-function">
  <marc:leader>     nam a22     uu 4500</marc:leader>
  <marc:controlfield tag="001">idp140215968406280</marc:controlfield>
  <marc:controlfield tag="003">DLC</marc:controlfield>
  <marc:controlfield tag="008">700101|||||||||xx            ||| |     |</marc:controlfield>
  <marc:datafield tag="040" ind1=" " ind2=" "/>
  <marc:datafield tag="245" xml:lang="eng" ind1="1" ind2="0">
    <marc:subfield code="a">حقول عيسى حسن الياسري</marc:subfield>
    <marc:subfield code="c">اعداد وتقديم فاطمه خليفه مؤذن</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="246" xml:lang="eng" ind1="1" ind2="1">
    <marc:subfield code="a">Fields, Issa Hassan al-Yasiri</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="250" xml:lang="eng" ind1=" " ind2=" ">
    <marc:subfield code="a">الطبعه الأولى</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="264" ind1="

In [13]:
print(stanford_graph.serialize(format='pretty-xml').decode())

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:bflc="http://id.loc.gov/ontologies/bflc/"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:bf="http://id.loc.gov/ontologies/bibframe/"
  xmlns:sinopia="http://sinopia.io/vocabulary/"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <bf:Instance rdf:about="https://trellis.stage.sinopia.io/repository/stanford/63f532de-65d3-4da4-bae3-9e981bd12eaf">
    <bf:date xml:lang="eng">2018</bf:date>
    <bf:provisionActivityStatement xml:lang="eng">بيروت : المؤسسه العربيه للدراسات والنشر، 2018</bf:provisionActivityStatement>
    <bf:editionStatement xml:lang="eng">الطبعه الأولى</bf:editionStatement>
    <bf:instanceOf>
      <bf:Work rdf:about="https://trellis.stage.sinopia.io/repository/stanford/2344e198-4306-45eb-ac55-e8a01836820b">
        <bf:subject rdf:resource="http://id.loc.gov/authorities/subjects/sh99005576"/>
        <bf:subject rdf:resource="http://id.loc.gov/authorities/names/no93018312"/>
        <bf:con

Last week noticed that the RDF XML produced from a combined graph of the BF Work and BF Instance results in the BF Instance `bf:instanceOf` to have as a child element the complete `bf:Work`. However, the `bibframe2marc` transformation expects the `bf:Work` to be at the same level as the `bf:Instance` as evidenced by the the sample XML from the Library of Congress. 

To replicate this XML structure, the BF Work and BF Instance will be serialized as XML first and then using xpath create a combined document with both of these XML elements at the top-level.

In [16]:
work_graph = triples2record(['https://trellis.stage.sinopia.io/repository/stanford/2344e198-4306-45eb-ac55-e8a01836820b'])
instance_graph = triples2record(['https://trellis.stage.sinopia.io/repository/stanford/63f532de-65d3-4da4-bae3-9e981bd12eaf'])

record_xml = etree.XML(instance_graph.serialize(format='pretty-xml'))

In [24]:
work_xml = etree.XML(work_graph.serialize(format='pretty-xml'))
work_element = work_xml.find("{http://id.loc.gov/ontologies/bibframe/}Work")

In [28]:
for child in work_xml.iterchildren():
    record_xml.append(child)

In [29]:
print(etree.tostring(record_xml).decode())

<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/">
  <bf:Instance rdf:about="https://trellis.stage.sinopia.io/repository/stanford/63f532de-65d3-4da4-bae3-9e981bd12eaf">
    <bf:provisionActivity>
      <bf:Publication rdf:nodeID="f3178cf27260e4153b6c4eeb5f81ea3bdb2">
        <bf:date xml:lang="eng">2018</bf:date>
        <bf:place rdf:resource="http://id.loc.gov/authorities/names/n83068103"/>
        <bf:agent>
          <bf:Agent rdf:nodeID="f3178cf27260e4153b6c4eeb5f81ea3bdb3">
            <rdfs:label xml:lang="eng">Mu&#700;assasah al-&#699;Arabi&#772;yah lil-Dira&#772;sa&#772;t wa-al-Nashr</rdfs:label>
          </bf:Agent>
        </bf:agent>
      </bf:Publication>
    </bf:provisionActivity>
    <bf:provisionActivityStatement xml:lang="eng">&#1576;&#1610;&#1585;&#1608;&#157

In [32]:
marc_xml = bf2marc_transform(record_xml)
marc21 = marcXMLto21(marc_xml)

In [33]:
print(marc21)

=LDR       nam a22     uu 4500
=001  idp140215949784616
=003  DLC
=008  700101|||||||||xx\\\\\\\\\\\\|||\|\ara\|
=040  \\
=050  \0$aPJ7874.A78$bZ76 2018
=522  \\
=100  1\$4http://id.loc.gov/vocabulary/relators/aut
=245  10$aحقول عيسى حسن الياسري$cاعداد وتقديم فاطمه خليفه مؤذن
=246  11$aFields, Issa Hassan al-Yasiri
=250  \\$aالطبعه الأولى
=264  \1$bMuʼassasah al-ʻArabīyah lil-Dirāsāt wa-al-Nashr,$c2018.
=300  \\$a304 pages$c24 cm
=653  \\$aArabic poetry--20th century--History and criticism
=653  \\$aCriticism and interpretation
=653  \\$aArabic poetry--Iraq--History and criticism
=653  \\$aYāsirī, ʻĪsá Ḥasan
=884  \\$aDLC bibframe2marc-xsl v0.1.0-SNAPSHOT$g20191216103247.0$qDLC$uhttps://github.com/lcnetdev/bibframe2marc-xsl



### Nested Work Example 2

In [52]:
s_nested2 = triples2record(['https://trellis.stage.sinopia.io/repository/stanford/3897dde4-0393-4600-a0da-cdd7ca0fe521'])
print(s_nested2.serialize(format='turtle').decode())
s_nested2_xml = graph2marcXML(s_nested2)
s_nested2_mrc21 = marcXMLto21(s_nested2_xml)
print(s_nested2_mrc21)

@prefix acl: <http://www.w3.org/ns/auth/acl#> .
@prefix as: <https://www.w3.org/ns/activitystreams#> .
@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix memento: <http://mementoweb.org/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://trellis.stage.sinopia.io/reposi

In [55]:
work_unnested_3 = triples2record(['https://trellis.stage.sinopia.io/repository/stanford/9508a59e-232b-47ca-901a-2982b3e89013',
                                  'https://trellis.stage.sinopia.io/repository/stanford/a6f2db0c-2162-4a46-9c41-878240ae0d36'])
print(work_unnested_3.serialize(format='turtle').decode())

@prefix acl: <http://www.w3.org/ns/auth/acl#> .
@prefix as: <https://www.w3.org/ns/activitystreams#> .
@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix memento: <http://mementoweb.org/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://id.loc.gov/authorities/genreForm

In [87]:
mod_stanford = rdflib.Graph()
init_ns(mod_stanford)
mod_stanford.parse(data="""@prefix acl: <http://www.w3.org/ns/auth/acl#> .
@prefix as: <https://www.w3.org/ns/activitystreams#> .
@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix memento: <http://mementoweb.org/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://id.loc.gov/authorities/genreForms/gf2014026415> rdfs:label "Literature" .

<http://id.loc.gov/authorities/names/n79007751> rdfs:label "New York (N.Y.)" .

<http://id.loc.gov/authorities/names/n79144795> rdfs:label "Ernst, Max, 1891-1976" .

<http://id.loc.gov/authorities/names/n80044679> a bf:Person, bf:Agent;
  rdfs:label "Éluard, Paul, 1895-1952" .

<http://id.loc.gov/authorities/names/n93122836> rdfs:label "Black Sun Press" .

<http://id.loc.gov/authorities/names/no93035745> a bf:Person, bf:Agent;
 rdfs:label "Chisholm, Hugh, 1913-1972" .

<http://id.loc.gov/authorities/subjects/sh85130846> rdfs:label "Surrealism" .

<http://id.loc.gov/vocabulary/carriers/nc> rdfs:label "volume" .

<http://id.loc.gov/vocabulary/contentTypes/txt> rdfs:label "text" .

<http://id.loc.gov/vocabulary/descriptionConventions/rda> rdfs:label "Resource Description and Access" .

<http://id.loc.gov/vocabulary/issuance/mono> rdfs:label "single unit" .

<http://id.loc.gov/vocabulary/mediaTypes/n> rdfs:label "unmediated" .

<http://id.loc.gov/vocabulary/millus/ill> rdfs:label "Illustrations" .

<http://id.loc.gov/vocabulary/relators/art> rdfs:label "Artist" .

<http://id.loc.gov/vocabulary/relators/aut> rdfs:label "Author" .

<http://id.loc.gov/vocabulary/relators/trl> rdfs:label "Translator" .

<http://vocab.getty.edu/aat/300123016> rdfs:label "artists' books (books)" .

<https://trellis.stage.sinopia.io/repository/stanford/9508a59e-232b-47ca-901a-2982b3e89013> a bf:Work ;
    bf:classification [ a bf:ClassificationLcc ;
            bf:classificationPortion "PQ2609.L75"@eng ;
            bf:itemPortion "M313 1943"@eng ] ;
    bf:content <http://id.loc.gov/vocabulary/contentTypes/txt> ;
    bf:contribution [ a bf:Contribution, bflc:PrimaryContribution;
            bf:agent <http://id.loc.gov/authorities/names/n80044679> ;
            bf:role <http://id.loc.gov/vocabulary/relators/aut> ] ;
    bf:genreForm <http://id.loc.gov/authorities/genreForms/gf2014026415>,
        <http://vocab.getty.edu/aat/300123016> ;
    bf:hasInstance <https://trellis.stage.sinopia.io/repository/stanford/a6f2db0c-2162-4a46-9c41-878240ae0d36> ;
    bf:illustrativeContent <http://id.loc.gov/vocabulary/millus/ill> ;
    bf:language <http://id.loc.gov/vocabulary/languages/eng> ;
    bf:subject <http://id.loc.gov/authorities/subjects/sh85130846> ;
    bf:title [ a bf:Title ;
            bf:mainTitle "Malheurs des immortels. English"@eng ] ;
    sinopia:hasResourceTemplate "ld4p:RT:bf2:Monograph:Work:Un-nested" .

<https://trellis.stage.sinopia.io/repository/stanford/a6f2db0c-2162-4a46-9c41-878240ae0d36> a bf:Instance ;
    bf:adminMetadata [ a bf:AdminMetadata ;
            bflc:catalogerId "kayteel"@eng ;
            bf:descriptionConventions <http://id.loc.gov/vocabulary/descriptionConventions/rda> ;
            bf:descriptionLanguage <http://id.loc.gov/vocabulary/languages/eng> ;
            bf:source <http://id.loc.gov/vocabulary/organizations/cst> ] ;
    bf:carrier <http://id.loc.gov/vocabulary/carriers/nc> ;
    bf:date "1943"@eng ;
    bf:dimensions "26 cm"@eng ;
    bf:extent [ a bf:Extent ;
            rdfs:label "44 pages, 7 unnumbered pages, 3 unnumbered pages of plates"@eng ] ;
    bf:instanceOf <https://trellis.stage.sinopia.io/repository/stanford/9508a59e-232b-47ca-901a-2982b3e89013> ;
    bf:issuance <http://id.loc.gov/vocabulary/issuance/mono> ;
    bf:media <http://id.loc.gov/vocabulary/mediaTypes/n> ;
    bf:note [ a bf:Note ;
            rdfs:label "Limited to 610 copies.--Colophon"@eng ],
        [ a bf:Note ;
            rdfs:label "English translation at top of page, French text below"@eng ] ;
    bf:provisionActivity [ a bf:Publication ;
            bf:agent [ a bf:Agent ;
                    bf:Agent <http://id.loc.gov/authorities/names/n93122836> ] ;
            bf:date "1943"@eng ;
            bf:place <http://id.loc.gov/authorities/names/n79007751> ] ;
    bf:provisionActivityStatement "[New York] : The Black Sun Press, 1943"@eng ;
    bf:responsibilityStatement "Max Ernst, Paul Eluard ; translated by Hugh Chisholm"@eng ;
    bf:title [ a bf:Title ;
            bf:mainTitle "Misfortunes of the immortals"@eng ] ;
    sinopia:hasResourceTemplate "ld4p:RT:bf2:Monograph:Instance:Un-nested" .

<http://id.loc.gov/vocabulary/languages/eng> rdfs:label "English" .""", format='turtle')

<Graph identifier=Nbd8c1576aa4148ccaf0b8142016a4c5c (<class 'rdflib.graph.Graph'>)>

In [104]:
print(mod_stanford.serialize(format='pretty-xml').decode())

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:bf="http://id.loc.gov/ontologies/bibframe/"
  xmlns:bflc="http://id.loc.gov/ontologies/bflc/"
  xmlns:sinopia="http://sinopia.io/vocabulary/"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://id.loc.gov/authorities/names/n79144795">
    <rdfs:label>Ernst, Max, 1891-1976</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://id.loc.gov/vocabulary/relators/trl">
    <rdfs:label>Translator</rdfs:label>
  </rdf:Description>
  <bf:Person rdf:about="http://id.loc.gov/authorities/names/no93035745">
    <rdfs:label>Chisholm, Hugh, 1913-1972</rdfs:label>
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Agent"/>
  </bf:Person>
  <rdf:Description rdf:about="http://id.loc.gov/vocabulary/relators/art">
    <rdfs:label>Artist</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://id.loc.gov/authori

In [100]:
mod2_xml = etree.XML("""<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:bf="http://id.loc.gov/ontologies/bibframe/"
  xmlns:bflc="http://id.loc.gov/ontologies/bflc/"
  xmlns:sinopia="http://sinopia.io/vocabulary/"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://id.loc.gov/authorities/names/n79144795">
    <rdfs:label>Ernst, Max, 1891-1976</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://id.loc.gov/vocabulary/relators/trl">
    <rdfs:label>Translator</rdfs:label>
  </rdf:Description>
  <bf:Person rdf:about="http://id.loc.gov/authorities/names/no93035745">
    <rdfs:label>Chisholm, Hugh, 1913-1972</rdfs:label>
    <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Agent"/>
  </bf:Person>
  <rdf:Description rdf:about="http://id.loc.gov/vocabulary/relators/art">
    <rdfs:label>Artist</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://id.loc.gov/authorities/subjects/sh85130846">
    <rdfs:label>Surrealism</rdfs:label>
  </rdf:Description>
  <bf:Instance rdf:about="https://trellis.stage.sinopia.io/repository/stanford/a6f2db0c-2162-4a46-9c41-878240ae0d36">
    <bf:carrier>
      <rdf:Description rdf:about="http://id.loc.gov/vocabulary/carriers/nc">
        <rdfs:label>volume</rdfs:label>
      </rdf:Description>
    </bf:carrier>
    <bf:issuance>
      <rdf:Description rdf:about="http://id.loc.gov/vocabulary/issuance/mono">
        <rdfs:label>single unit</rdfs:label>
      </rdf:Description>
    </bf:issuance>
    <bf:instanceOf>https://trellis.stage.sinopia.io/repository/stanford/9508a59e-232b-47ca-901a-2982b3e89013</bf:instanceOf>
    <bf:provisionActivity>
      <bf:Publication rdf:nodeID="ub55bL92C26">
        <bf:date xml:lang="eng">1943</bf:date>
        <bf:agent>
            <bf:Agent rdf:resource="http://id.loc.gov/authorities/names/n80044679">
               <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Person" />

              <rdfs:label>Éluard, Paul, 1895-1952</rdfs:label>
            </bf:Agent>
        </bf:agent>
        <bf:place rdf:resource="http://id.loc.gov/authorities/names/n79007751"/>
      </bf:Publication>
    </bf:provisionActivity>
    <bf:title>
      <bf:Title rdf:nodeID="ub55bL99C14">
        <bf:mainTitle xml:lang="eng">Misfortunes of the immortals</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:provisionActivityStatement xml:lang="eng">[New York] : The Black Sun Press, 1943</bf:provisionActivityStatement>
    <bf:note>
      <bf:Note rdf:nodeID="ub55bL90C9">
        <rdfs:label xml:lang="eng">English translation at top of page, French text below</rdfs:label>
      </bf:Note>
    </bf:note>
    <bf:media>
      <rdf:Description rdf:about="http://id.loc.gov/vocabulary/mediaTypes/n">
        <rdfs:label>unmediated</rdfs:label>
      </rdf:Description>
    </bf:media>
    <bf:extent>
      <bf:Extent rdf:nodeID="ub55bL83C15">
        <rdfs:label xml:lang="eng">44 pages, 7 unnumbered pages, 3 unnumbered pages of plates</rdfs:label>
      </bf:Extent>
    </bf:extent>
    <bf:dimensions xml:lang="eng">26 cm</bf:dimensions>
    <bf:date xml:lang="eng">1943</bf:date>
    <bf:note>
      <bf:Note rdf:nodeID="ub55bL88C13">
        <rdfs:label xml:lang="eng">Limited to 610 copies.--Colophon</rdfs:label>
      </bf:Note>
    </bf:note>
    <bf:responsibilityStatement xml:lang="eng">Max Ernst, Paul Eluard ; translated by Hugh Chisholm</bf:responsibilityStatement>
    <bf:adminMetadata>
      <bf:AdminMetadata rdf:nodeID="ub55bL75C22">
        <bf:descriptionConventions rdf:resource="http://id.loc.gov/vocabulary/descriptionConventions/rda"/>
        <bf:source rdf:resource="http://id.loc.gov/vocabulary/organizations/cst"/>
        <bf:descriptionLanguage rdf:resource="http://id.loc.gov/vocabulary/languages/eng"/>
        <bflc:catalogerId xml:lang="eng">kayteel</bflc:catalogerId>
      </bf:AdminMetadata>
    </bf:adminMetadata>
    <sinopia:hasResourceTemplate>ld4p:RT:bf2:Monograph:Instance:Un-nested</sinopia:hasResourceTemplate>
  </bf:Instance>
  <bf:Work rdf:about="https://trellis.stage.sinopia.io/repository/stanford/9508a59e-232b-47ca-901a-2982b3e89013">
        <bf:title>
          <bf:Title rdf:nodeID="ub55bL70C14">
            <bf:mainTitle xml:lang="eng">Malheurs des immortels. English</bf:mainTitle>
          </bf:Title>
        </bf:title>
        <sinopia:hasResourceTemplate>ld4p:RT:bf2:Monograph:Work:Un-nested</sinopia:hasResourceTemplate>
        <bf:content rdf:resource="http://id.loc.gov/vocabulary/contentTypes/txt"/>
        <bf:illustrativeContent rdf:resource="http://id.loc.gov/vocabulary/millus/ill"/>
        <bf:genreForm rdf:resource="http://vocab.getty.edu/aat/300123016"/>
        <bf:contribution>
          <bflc:PrimaryContribution rdf:nodeID="ub55bL61C21">
            <rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Contribution"/>
            <bf:role rdf:resource="http://id.loc.gov/vocabulary/relators/aut"/>
            <bf:agent rdf:resource="http://id.loc.gov/authorities/names/n80044679"/>
          </bflc:PrimaryContribution>
        </bf:contribution>
        <bf:classification>
          <bf:ClassificationLcc rdf:nodeID="ub55bL57C23">
            <bf:itemPortion xml:lang="eng">M313 1943</bf:itemPortion>
            <bf:classificationPortion xml:lang="eng">PQ2609.L75</bf:classificationPortion>
          </bf:ClassificationLcc>
        </bf:classification>
        <bf:language rdf:resource="http://id.loc.gov/vocabulary/languages/eng"/>
        <bf:subject rdf:resource="http://id.loc.gov/authorities/subjects/sh85130846"/>
        <bf:hasInstance rdf:resource="https://trellis.stage.sinopia.io/repository/stanford/a6f2db0c-2162-4a46-9c41-878240ae0d36"/>
        <bf:genreForm rdf:resource="http://id.loc.gov/authorities/genreForms/gf2014026415"/>
      </bf:Work>
  <rdf:Description rdf:about="http://id.loc.gov/vocabulary/languages/eng">
    <rdfs:label>English</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://vocab.getty.edu/aat/300123016">
    <rdfs:label>artists' books (books)</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://id.loc.gov/authorities/genreForms/gf2014026415">
    <rdfs:label>Literature</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://id.loc.gov/vocabulary/relators/aut">
    <rdfs:label>Author</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://id.loc.gov/vocabulary/descriptionConventions/rda">
    <rdfs:label>Resource Description and Access</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://id.loc.gov/authorities/names/n93122836">
    <rdfs:label>Black Sun Press</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://id.loc.gov/authorities/names/n79007751">
    <rdfs:label>New York (N.Y.)</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://id.loc.gov/vocabulary/contentTypes/txt">
    <rdfs:label>text</rdfs:label>
  </rdf:Description>
  <rdf:Description rdf:about="http://id.loc.gov/vocabulary/millus/ill">
    <rdfs:label>Illustrations</rdfs:label>
  </rdf:Description>
</rdf:RDF>""".encode())

In [101]:
marc_xml = bf2marc_transform(mod2_xml)

In [102]:
print(marc_xml)

<?xml version="1.0"?>
<marc:record xmlns:marc="http://www.loc.gov/MARC21/slim" xmlns:fn="http://www.w3.org/2005/xpath-function">
  <marc:leader>     nam a22     uu 4500</marc:leader>
  <marc:controlfield tag="001">idp140491413088744</marc:controlfield>
  <marc:controlfield tag="003">DLC</marc:controlfield>
  <marc:controlfield tag="008">19121315|||||||||xx            ||| | eng |</marc:controlfield>
  <marc:datafield tag="040" ind1=" " ind2=" "/>
  <marc:datafield tag="050" ind1=" " ind2="0">
    <marc:subfield code="a">PQ2609.L75</marc:subfield>
    <marc:subfield code="b">M313 1943</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="245" xml:lang="eng" ind1="1" ind2="0">
    <marc:subfield code="a">Misfortunes of the immortals</marc:subfield>
    <marc:subfield code="c">Max Ernst, Paul Eluard ; translated by Hugh Chisholm</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="264" ind1=" " ind2="1">
    <marc:subfield code="c">1943.</marc:subfield>
  </marc:datafield>
  <ma

In [96]:
marc_xml = graph2marcXML(mod_stanford)

In [97]:
print(marc_xml)

<?xml version="1.0"?>
<marc:record xmlns:marc="http://www.loc.gov/MARC21/slim" xmlns:fn="http://www.w3.org/2005/xpath-function">
  <marc:leader>     nam a22     uu 4500</marc:leader>
  <marc:controlfield tag="001">idp140491291601576</marc:controlfield>
  <marc:controlfield tag="003">DLC</marc:controlfield>
  <marc:controlfield tag="008">19121314|||||||||xx            ||| |     |</marc:controlfield>
  <marc:datafield tag="040" ind1=" " ind2=" "/>
  <marc:datafield tag="245" xml:lang="eng" ind1="1" ind2="0">
    <marc:subfield code="a">Misfortunes of the immortals</marc:subfield>
    <marc:subfield code="c">Max Ernst, Paul Eluard ; translated by Hugh Chisholm</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="264" ind1=" " ind2="1">
    <marc:subfield code="c">1943.</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="300" xml:lang="eng" ind1=" " ind2=" ">
    <marc:subfield code="a">44 pages, 7 unnumbered pages, 3 unnumbered pages of plates</marc:subfield>
    <marc:subfie

# Example -- University of California Davis
Repeat the steps in example one to create a combined BF Work and Instance Graph. Also printing out the number of triples in each graph after it is parsed into the final graph. In the resulting serialization in Turtle you can see the BF Work and Instance is represented as top-level subject edges. The Trellis GET response text was encoded in **ISO-8859-1** and needed to be decoded to **utf-8**.

In [6]:
davis_record = triples2record(["https://trellis.stage.sinopia.io/repository/ucdavis/be99291f-ff37-4dd4-bc71-8ff4ac46d3fb",
                               "https://trellis.stage.sinopia.io/repository/ucdavis/307b37cd-980a-436c-881b-6ed1aa0afeeb"])


print(davis_record.serialize(format='turtle').decode())

@prefix acl: <http://www.w3.org/ns/auth/acl#> .
@prefix as: <https://www.w3.org/ns/activitystreams#> .
@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix memento: <http://mementoweb.org/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://trellis.stage.sinopia.io/reposi

### RDF-XML Serialization and MARC Transformation
With the new RDF graph called `davis_record`, we can then serialize it as RDF-XML and pass it through `bf2marc_transform` XSLT to produce the MARC XML in `davis_marc_xml`.

In [12]:
davis_marc_xml = graph2marcXML(davis_record)
print(davis_marc_xml)

<?xml version="1.0"?>
<marc:record xmlns:marc="http://www.loc.gov/MARC21/slim" xmlns:fn="http://www.w3.org/2005/xpath-function">
  <marc:leader>     nam a22     uu 4500</marc:leader>
  <marc:controlfield tag="001">idp140491395026712</marc:controlfield>
  <marc:controlfield tag="003">DLC</marc:controlfield>
  <marc:controlfield tag="008">19121310|||||||||xx            ||| |     |</marc:controlfield>
  <marc:datafield tag="020" ind1=" " ind2=" ">
    <marc:subfield code="a">9787502744878</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="245" xml:lang="chi" ind1="1" ind2="0">
    <marc:subfield code="a">中國古動物</marc:subfield>
    <marc:subfield code="c">關键</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="246" xml:lang="jpn" ind1="1" ind2="1">
    <marc:subfield code="a">中国古動物</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="246" xml:lang="eng" ind1="1" ind2="1">
    <marc:subfield code="a">The fossil animals of China</marc:subfield>
  </marc:datafield>
  <marc:

Use `pymarc` to create MARC21 format.

In [15]:
marc_record = marcXMLto21(davis_marc_xml)
print(marc_record)

=LDR       nam a22     uu 4500
=001  idp140491395026712
=003  DLC
=008  19121310|||||||||xx\\\\\\\\\\\\|||\|\\\\\|
=020  \\$a9787502744878
=245  10$a中國古動物$c關键
=246  11$a中国古動物
=246  11$aThe fossil animals of China
=250  \\$a第1版
=264  \1$c1998.
=300  \\$a197 pages$c29 cm
=500  \\$aText in Chinese, English, and Japanese ; nomenclature also in Latin
=500  \\$aIncludes index in Latin
=884  \\$aDLC bibframe2marc-xsl v0.1.0-SNAPSHOT$g20191213104714.0$qDLC$uhttps://github.com/lcnetdev/bibframe2marc-xsl



## Example from Library of Congress
Copied the BIBFRAME RDF-XML from the comparsion tool-kit at (http://id.loc.gov/tools/bibframe/comparebf-lccn/2004701771.xml) and directly parsed and transformed the XML to MARC.

In [56]:

raw_lcc_record = """<rdf:RDF xmlns:bf = "http://id.loc.gov/ontologies/bibframe/" xmlns:bflc = "http://id.loc.gov/ontologies/bflc/" xmlns:madsrdf = "http://www.loc.gov/mads/rdf/v1#" xmlns:rdf = "http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:rdfs = "http://www.w3.org/2000/01/rdf-schema#" >
<bf:Work rdf:about="http://bibframe.example.org/13678831#Work" >
<bf:adminMetadata >
<bf:AdminMetadata >
<bf:generationProcess >
<bf:GenerationProcess >
<rdfs:label >DLC marc2bibframe2 v1.5.1: 2019-12-12T11:11:08-05:00</rdfs:label>
</bf:GenerationProcess>
</bf:generationProcess>
<bf:status >
<bf:Status >
<rdfs:label >corrected or revised</rdfs:label>
<bf:code >c</bf:code>
</bf:Status>
</bf:status>
<bflc:encodingLevel >
<bflc:EncodingLevel rdf:about="http://id.loc.gov/vocabulary/menclvl/4" >
<rdfs:label >core</rdfs:label>
</bflc:EncodingLevel>
</bflc:encodingLevel>
<bf:descriptionConventions >
<bf:DescriptionConventions rdf:about="http://id.loc.gov/vocabulary/descriptionConventions/aacr" >
<rdfs:label >aacr</rdfs:label>
</bf:DescriptionConventions>
</bf:descriptionConventions>
<bf:identifiedBy >
<bf:Local >
<rdf:value >13678831</rdf:value>
<bf:assigner >
<bf:Agent rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" />
</bf:assigner>
</bf:Local>
</bf:identifiedBy>
<bf:changeDate rdf:datatype="http://www.w3.org/2001/XMLSchema#dateTime" >2007-08-25T03:55:05</bf:changeDate>
<bf:creationDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date" >2004-06-18</bf:creationDate>
<bf:source >
<bf:Source >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Agent" />
<rdfs:label >DLC-R</rdfs:label>
</bf:Source>
</bf:source>
<bf:source >
<bf:Source >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Agent" />
<rdfs:label >DLC-R</rdfs:label>
</bf:Source>
</bf:source>
<bf:descriptionModifier >
<bf:Agent rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" >
<rdfs:label >DLC</rdfs:label>
</bf:Agent>
</bf:descriptionModifier>
<bf:descriptionAuthentication >
<bf:DescriptionAuthentication rdf:about="http://id.loc.gov/vocabulary/marcauthen/pcc" >
<rdf:value >pcc</rdf:value>
</bf:DescriptionAuthentication>
</bf:descriptionAuthentication>
</bf:AdminMetadata>
</bf:adminMetadata>
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Text" />
<bf:language >
<bf:Language rdf:about="http://id.loc.gov/vocabulary/languages/jpn" />
</bf:language>
<bf:illustrativeContent >
<bf:Illustration rdf:about="http://id.loc.gov/vocabulary/millus/ill" >
<rdfs:label >illustrations</rdfs:label>
</bf:Illustration>
</bf:illustrativeContent>
<bf:supplementaryContent >
<bf:SupplementaryContent rdf:about="http://id.loc.gov/authorities/genreForms/gf2014026048" >
<rdfs:label >bibliographies</rdfs:label>
</bf:SupplementaryContent>
</bf:supplementaryContent>
<bf:supplementaryContent >
<bf:SupplementaryContent rdf:about="http://id.loc.gov/vocabulary/msupplcont/index" >
<rdfs:label >Index present</rdfs:label>
</bf:SupplementaryContent>
</bf:supplementaryContent>
<bf:identifiedBy >
<bf:Lccn >
<rdf:value >2004701771</rdf:value>
</bf:Lccn>
</bf:identifiedBy>
<bf:geographicCoverage >
<bf:GeographicCoverage rdf:about="http://id.loc.gov/vocabulary/geographicAreas/a-ja" />
</bf:geographicCoverage>
<bf:classification >
<bf:ClassificationLcc >
<bf:source >
<bf:Source rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" />
</bf:source>
<bf:classificationPortion >DS859</bf:classificationPortion>
<bf:itemPortion >.T3626 2004</bf:itemPortion>
</bf:ClassificationLcc>
</bf:classification>
<bf:classification >
<bf:Classification >
<bf:classificationPortion >210.42</bf:classificationPortion>
<bf:source >
<bf:Source >
<rdfs:label >njb/9</rdfs:label>
</bf:Source>
</bf:source>
</bf:Classification>
</bf:classification>
<bf:contribution >
<bf:Contribution >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bflc/PrimaryContribution" />
<bf:agent >
<bf:Agent rdf:about="http://id.loc.gov/authorities/names/nr2004032488" >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Person" />
<bflc:name00MatchKey >Takahashi, Kazuki, 1967-</bflc:name00MatchKey>
<bflc:primaryContributorName00MatchKey >Takahashi, Kazuki, 1967-</bflc:primaryContributorName00MatchKey>
<bflc:name00MarcKey >1001 $6880-01$aTakahashi, Kazuki,$d1967-</bflc:name00MarcKey>
<rdfs:label >Takahashi, Kazuki, 1967-</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:role >
<bf:Role rdf:about="http://id.loc.gov/vocabulary/relators/ctb" />
</bf:role>
</bf:Contribution>
</bf:contribution>
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bf:title >
<bf:Title >
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bflc:titleSortKey >Chūsei shōensei to Kamakura Bakufu</bflc:titleSortKey>
<bf:mainTitle >Chūsei shōensei to Kamakura Bakufu</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:supplementaryContent >
<bf:SupplementaryContent >
<rdfs:label >Includes bibliographical references and indexes.</rdfs:label>
</bf:SupplementaryContent>
</bf:supplementaryContent>
<bf:subject >
<bf:Place rdf:about="http://id.loc.gov/authorities/subjects/sh85069454" >
<rdf:type rdf:resource="http://www.loc.gov/mads/rdf/v1#ComplexSubject" />
<rdfs:label >Japan--History--Kamakura period, 1185-1333.</rdfs:label>
<madsrdf:authoritativeLabel >Japan--History--Kamakura period, 1185-1333.</madsrdf:authoritativeLabel>
<madsrdf:isMemberOfMADSScheme rdf:resource="http://id.loc.gov/authorities/subjects" />
<madsrdf:componentList rdf:parseType="Collection" >
<madsrdf:Geographic rdf:about="http://id.loc.gov/authorities/names/n78089021" >
<madsrdf:authoritativeLabel >Japan</madsrdf:authoritativeLabel>
</madsrdf:Geographic>
<madsrdf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh85061212" >
<madsrdf:authoritativeLabel >History</madsrdf:authoritativeLabel>
</madsrdf:Topic>
<madsrdf:Temporal >
<madsrdf:authoritativeLabel >Kamakura period, 1185-1333</madsrdf:authoritativeLabel>
</madsrdf:Temporal>
</madsrdf:componentList>
<bf:source >
<bf:Source >
<bf:code >lcsh</bf:code>
</bf:Source>
</bf:source>
</bf:Place>
</bf:subject>
<bf:subject >
<bf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh2008107312" >
<rdf:type rdf:resource="http://www.loc.gov/mads/rdf/v1#ComplexSubject" />
<rdfs:label >Manors--Japan--History.</rdfs:label>
<madsrdf:authoritativeLabel >Manors--Japan--History.</madsrdf:authoritativeLabel>
<madsrdf:isMemberOfMADSScheme rdf:resource="http://id.loc.gov/authorities/subjects" />
<madsrdf:componentList rdf:parseType="Collection" >
<madsrdf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh85080612" >
<madsrdf:authoritativeLabel >Manors</madsrdf:authoritativeLabel>
</madsrdf:Topic>
<madsrdf:Geographic rdf:about="http://id.loc.gov/authorities/names/n78089021" >
<madsrdf:authoritativeLabel >Japan</madsrdf:authoritativeLabel>
</madsrdf:Geographic>
<madsrdf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh85061212" >
<madsrdf:authoritativeLabel >History</madsrdf:authoritativeLabel>
</madsrdf:Topic>
</madsrdf:componentList>
<bf:source >
<bf:Source >
<bf:code >lcsh</bf:code>
</bf:Source>
</bf:source>
</bf:Topic>
</bf:subject>
<bf:contribution >
<bf:Contribution >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bflc/PrimaryContribution" />
<bf:agent >
<bf:Agent rdf:about="http://id.loc.gov/authorities/names/nr2004032488" >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Person" />
<bflc:name00MatchKey >高橋一樹, 1967-</bflc:name00MatchKey>
<bflc:primaryContributorName00MatchKey >高橋一樹, 1967-</bflc:primaryContributorName00MatchKey>
<bflc:name00MarcKey >8801 $6100-01/$1$a高橋一樹,$d1967-</bflc:name00MarcKey>
<rdfs:label xml:lang="ja-jpan" >高橋一樹, 1967-</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:role >
<bf:Role rdf:about="http://id.loc.gov/vocabulary/relators/ctb" />
</bf:role>
</bf:Contribution>
</bf:contribution>
<bf:title >
<bf:Title >
<rdfs:label xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</rdfs:label>
<bflc:titleSortKey >中世荘園制と鎌倉幕府</bflc:titleSortKey>
<bf:mainTitle xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:hasInstance rdf:resource="http://bibframe.example.org/13678831#Instance" />
</bf:Work>
<bf:Instance rdf:about="http://bibframe.example.org/13678831#Instance" >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Print" />
<bf:issuance >
<bf:Issuance rdf:about="http://id.loc.gov/vocabulary/issuance/mono" />
</bf:issuance>
<bf:provisionActivity >
<bf:ProvisionActivity >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Publication" />
<bf:date rdf:datatype="http://id.loc.gov/datatypes/edtf" >2004</bf:date>
<bf:place >
<bf:Place rdf:about="http://id.loc.gov/vocabulary/countries/ja" />
</bf:place>
</bf:ProvisionActivity>
</bf:provisionActivity>
<bf:identifiedBy >
<bf:Lccn >
<rdf:value >2004701771</rdf:value>
</bf:Lccn>
</bf:identifiedBy>
<bf:identifiedBy >
<bf:Isbn >
<rdf:value >4827311838</rdf:value>
</bf:Isbn>
</bf:identifiedBy>
<bf:identifiedBy >
<bf:Local >
<rdf:value >DCLP04-B11050</rdf:value>
<bf:source >
<bf:Source >
<rdfs:label >CStRLIN</rdfs:label>
</bf:Source>
</bf:source>
</bf:Local>
</bf:identifiedBy>
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bf:title >
<bf:Title >
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bflc:titleSortKey >Chūsei shōensei to Kamakura Bakufu</bflc:titleSortKey>
<bf:mainTitle >Chūsei shōensei to Kamakura Bakufu</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:responsibilityStatement >Takahashi Kazuki cho</bf:responsibilityStatement>
<bf:provisionActivity >
<bf:ProvisionActivity >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Publication" />
<bf:place >
<bf:Place >
<rdfs:label >Tōkyō</rdfs:label>
</bf:Place>
</bf:place>
<bf:agent >
<bf:Agent >
<rdfs:label >Hanawa Shobō</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:date >2004</bf:date>
</bf:ProvisionActivity>
</bf:provisionActivity>
<bf:provisionActivityStatement >Tōkyō : Hanawa Shobō, 2004.</bf:provisionActivityStatement>
<bf:extent >
<bf:Extent >
<rdfs:label >vii, 433, 31 p.</rdfs:label>
</bf:Extent>
</bf:extent>
<bf:note >
<bf:Note >
<bf:noteType >Physical details</bf:noteType>
<rdfs:label >ill.</rdfs:label>
</bf:Note>
</bf:note>
<bf:dimensions >22 cm.</bf:dimensions>
<bf:title >
<bf:Title >
<rdfs:label xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</rdfs:label>
<bflc:titleSortKey >中世荘園制と鎌倉幕府</bflc:titleSortKey>
<bf:mainTitle xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:responsibilityStatement xml:lang="ja-jpan" >高橋一樹著</bf:responsibilityStatement>
<bf:provisionActivity >
<bf:ProvisionActivity >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Publication" />
<bf:place >
<bf:Place >
<rdfs:label xml:lang="ja-jpan" >東京</rdfs:label>
</bf:Place>
</bf:place>
<bf:agent >
<bf:Agent >
<rdfs:label xml:lang="ja-jpan" >塙書房</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:date xml:lang="ja-jpan" >2004</bf:date>
</bf:ProvisionActivity>
</bf:provisionActivity>
<bf:provisionActivityStatement xml:lang="ja-jpan" >東京 : 塙書房, 2004.</bf:provisionActivityStatement>
<bf:instanceOf rdf:resource="http://bibframe.example.org/13678831#Work" />
<bf:hasItem >
<bf:Item rdf:about="http://bibframe.example.org/13678831#Item050-14" >
<bf:shelfMark >
<bf:ShelfMarkLcc >
<rdfs:label >DS859.T3626 2004</rdfs:label>
<bf:source >
<bf:Source rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" />
</bf:source>
</bf:ShelfMarkLcc>
</bf:shelfMark>
<bf:itemOf rdf:resource="http://bibframe.example.org/13678831#Instance" />
</bf:Item>
</bf:hasItem>
</bf:Instance>
</rdf:RDF>"""

In [105]:
loc_graph = rdflib.Graph()
init_ns(loc_graph)
loc_graph.parse(data=raw_lcc_record, format='xml')
print(loc_graph.serialize(format='turtle').decode())

@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix madsrdf: <http://www.loc.gov/mads/rdf/v1#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://bibframe.example.org/13678831#Item050-14> a bf:Item ;
    bf:itemOf <http://bibframe.example.org/13678831#Instance> ;
    bf:shelfMark [ a bf:ShelfMarkLcc ;
            rdfs:label "DS859.T3626 2004" ;
            bf:source <http://id.loc.gov/vocabulary/organizations/dlc> ] .

<http://bibframe.example.org/13678831#Work> a bf:Text,
        bf:Work ;
    rdfs:label "Chūsei shōensei to Kamakura Bakufu" ;
    bf:adminMetadata [ a bf:AdminMetadata ;
            bflc:encodingLevel <http://id.loc.gov/vocabulary/menclvl/4> ;
            bf:changeDate "2007-08-

In [63]:
loc_xml = etree.XML(raw_lcc_record)
loc_marc = bf2marc_transform(loc_xml)

In [59]:
loc_marc21 = marcXMLto21(loc_marc)
print(loc_marc21)

=LDR       cam a22     4a 4500
=001  13678831
=003  DLC
=005  20070825035505.0
=008  040618s2004\\\\ja\\\\\\\\\\\\|||\|\jpn\|
=010  \\$a2004701771
=020  \\$a4827311838
=035  \\$a(CStRLIN)DCLP04-B11050
=040  \\$aDLC-R$cDLC-R$dDLC$eaacr
=042  \\$apcc
=043  \\$aa-ja---
=050  \0$aDS859$b.T3626 2004
=084  \\$a210.42$2njb/9
=100  1\$aTakahashi, Kazuki, 1967-$0http://id.loc.gov/authorities/names/nr2004032488$4http://id.loc.gov/vocabulary/relators/ctb
=240  10$aChūsei shōensei to Kamakura Bakufu
=241  10$aChūsei shōensei to Kamakura Bakufu
=245  10$a中世荘園制と鎌倉幕府$c高橋一樹著
=264  \1$ahttp://id.loc.gov/vocabulary/countries/ja$c2004.
=264  \1$aTōkyō:$bHanawa Shobō,$c2004.
=264  \1$a東京:$b塙書房,$c2004.
=300  \\$avii, 433, 31 p.$bill.$c22 cm.
=500  \\$aill.
=525  \\$abibliographies
=525  \\$aIndex present
=525  \\$aIncludes bibliographical references and indexes.
=651  \0$0http://id.loc.gov/authorities/subjects/sh85069454$aJapan$xHistory$0http://id.loc.gov/authorities/subjects/sh85061212$yKamakura period, 1